In [ ]:
# %% [markdown]
# # Lab 10: Integration - A Complete Basic System
#
# **Goal:** Combine the concepts from the previous labs—nodes, conditional routing, and ReAct-style steps—into a single, comprehensive recruitment processing system.
#
# ---

# %% [markdown]
# ## Setup
#
# We'll install our libraries and set up the state and nodes needed for our complete system.

# %%
# Install required packages
!pip install langgraph --quiet
print("Libraries installed.")

# Basic imports
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

# %% [markdown]
# ---
# ## Part A: The Comprehensive State
#
# Our final state needs to hold all the information we gather throughout the process, from initial parsing to the final recommendation.

# %%
class ComprehensiveRecruitmentState(TypedDict):
    # Basic Info
    candidate_name: str
    resume_text: str

    # Processing Results
    extracted_skills: List[str]
    experience_years: int
    technical_score: int

    # Verification & Reasoning
    verification_results: List[str]
    reasoning_steps: List[str]

    # Final Decision
    recommendation: str
    confidence_score: int

print("Comprehensive state defined.")

# %% [markdown]
# ---
# ## Part B: Building the Integrated Graph
#
# We will define all the nodes for our complete system and then wire them together.

# %%
# Define all nodes for the complete system

def initial_parsing(state: ComprehensiveRecruitmentState) -> dict:
    """Parses the resume to extract skills and experience."""
    print(f"---NODE: PARSING ({state['candidate_name']})---")
    resume = state["resume_text"].lower()

    # Extract skills
    skills = []
    skill_list = ["python", "java", "javascript", "management", "leadership", "sql"]
    for skill in skill_list:
        if skill in resume:
            skills.append(skill.title())

    # Extract experience
    experience = 0
    if "senior" in resume or "5+ years" in resume:
        experience = 5
    elif "3 years" in resume or "mid-level" in resume:
        experience = 3
    elif "junior" in resume or "1 year" in resume:
        experience = 1

    return {
        "extracted_skills": skills,
        "experience_years": experience,
        "reasoning_steps": [f"Parsed resume for {state['candidate_name']}"]
    }

def technical_assessment(state: ComprehensiveRecruitmentState) -> dict:
    """Calculates a technical score based on skills and experience."""
    print(f"---NODE: ASSESSING ({state['candidate_name']})---")
    skills = state["extracted_skills"]
    experience = state["experience_years"]

    skill_score = len(skills) * 15
    experience_score = experience * 10
    tech_score = min(skill_score + experience_score, 100)

    return {
        "technical_score": tech_score,
        "reasoning_steps": state["reasoning_steps"] + [f"Technical assessment score: {tech_score}/100"]
    }

def verification_step(state: ComprehensiveRecruitmentState) -> dict:
    """A ReAct-style step to simulate verifying the candidate's claims."""
    print(f"---NODE: VERIFYING ({state['candidate_name']})---")
    skills = state["extracted_skills"]

    verifications = []
    if "Python" in skills:
        verifications.append("Python skills verified via GitHub portfolio.")
    if "Management" in skills:
        verifications.append("Leadership claims require interview validation.")

    return {
        "verification_results": verifications,
        "reasoning_steps": state["reasoning_steps"] + [f"Completed {len(verifications)} verification checks."]
    }

def final_recommendation(state: ComprehensiveRecruitmentState) -> dict:
    """Makes the final hiring recommendation with a confidence score."""
    print(f"---NODE: RECOMMENDING ({state['candidate_name']})---")
    tech_score = state["technical_score"]
    verifications = len(state["verification_results"])

    confidence = min((verifications * 25) + (tech_score // 2), 100)

    if tech_score >= 70 and confidence >= 60:
        recommendation = "Strong Hire - Schedule Technical Interview"
    elif tech_score >= 50:
        recommendation = "Potential Hire - Requires Additional Assessment"
    else:
        recommendation = "Not Recommended - Skills Gap"

    return {
        "recommendation": recommendation,
        "confidence_score": confidence,
        "reasoning_steps": state["reasoning_steps"] + [f"Final recommendation: {recommendation}"]
    }

# Define the routing function for our conditional edge
def route_by_score(state: ComprehensiveRecruitmentState) -> str:
    """Routes based on the technical score."""
    print(f"---ROUTING ({state['candidate_name']})---")
    if state["technical_score"] >= 70:
        return "high_potential"
    else:
        return "standard_process"


# %% [markdown]
# ### Assembling the Complete Graph
#
# Now we wire all the nodes and the conditional router together.

# %%
# Build the complete workflow
workflow = StateGraph(ComprehensiveRecruitmentState)

# Add all nodes
workflow.add_node("parse", initial_parsing)
workflow.add_node("assess", technical_assessment)
workflow.add_node("verify", verification_step)
workflow.add_node("recommend", final_recommendation)

# Add edges
workflow.set_entry_point("parse")
workflow.add_edge("parse", "assess")

# Add conditional routing
workflow.add_conditional_edges(
    "assess",
    route_by_score,
    {
        # High potential candidates go directly to verification
        "high_potential": "verify",
        # Standard candidates go directly to the recommendation
        "standard_process": "recommend"
    }
)

# The verification path also leads to the final recommendation
workflow.add_edge("verify", "recommend")
workflow.add_edge("recommend", END)

# Compile the final application
complete_app = workflow.compile()
print("Complete recruitment agent compiled successfully!")


# %% [markdown]
# ---
# ## Part C: Testing the Complete System
#
# Let's run a few different candidate profiles through our system to see the entire process in action.

# %%
# Define test candidates
test_candidates = [
    {
        "name": "Alice Johnson",
        "resume": "Senior Python developer with 5+ years experience, leadership, and SQL skills."
    },
    {
        "name": "Bob Smith",
        "resume": "Junior developer with some Python experience, eager to learn."
    }
]

print("---STARTING COMPLETE RECRUITMENT SYSTEM TEST---\n")

for candidate in test_candidates:
    print(f"---Processing: {candidate['name']}---")

    # Set up the initial state for the invocation
    initial_state = {
        "candidate_name": candidate["name"],
        "resume_text": candidate["resume"],
    }

    # Run the graph for the candidate
    result = complete_app.invoke(initial_state)

    # Print a summary of the final result
    print(f"\n📋 FINAL SUMMARY FOR {result['candidate_name']}:")
    print(f"  - Technical Score: {result['technical_score']}/100")
    print(f"  - Recommendation: {result['recommendation']}")
    print(f"  - Confidence: {result['confidence_score']}%")
    print("  - Reasoning Process:")
    for step in result['reasoning_steps']:
        print(f"    - {step}")
    print("\n" + "="*50 + "\n")